# **Step 1:** Import drivers.csv, countries.csv, and nationality.csv to merge all three together for a cleaned dataframe with "country_name", "latitude", and "longtitude" #

In [1]:
# Importing dependencies
import pandas as pd

In [2]:
# Importing driver.csv data into a dataframe
drivers_df = pd.read_csv('Resources/drivers.csv')
drivers_df.head()

# Dropping "driverRef","number","code", and "url" columns from drivers_df
drivers_df = drivers_df[['driverId', 'forename', 'surname', 'dob', 'nationality']]
drivers_df.head()

# Changing column names to be more intuitive for drivers_df
drivers_df=drivers_df.rename(columns = {
                                        'driverId': 'driver_id', 
                                        'forename': 'first_name', 
                                        'surname': 'last_name'
                                        })
drivers_df

,driver_id,first_name,last_name,dob,nationality
0,1,Lewis,Hamilton,1985-01-07,British
1,2,Nick,Heidfeld,1977-05-10,German
2,3,Nico,Rosberg,1985-06-27,German
3,4,Fernando,Alonso,1981-07-29,Spanish
4,5,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...
854,856,Nyck,de Vries,1995-02-06,Dutch
855,857,Oscar,Piastri,2001-04-06,Australian
856,858,Logan,Sargeant,2000-12-31,American
857,859,Liam,Lawson,2002-02-11,New Zealander


In [3]:
# Finding all unique nationalities in drivers_df
unique_nationalities = drivers_df['nationality'].unique()

print(unique_nationalities)

['British' 'German' 'Spanish' 'Finnish' 'Japanese' 'French' 'Polish'
 'Brazilian' 'Italian' 'Australian' 'Austrian' 'American' 'Dutch'
 'Colombian' 'Portuguese' 'Canadian' 'Indian' 'Hungarian' 'Irish' 'Danish'
 'Argentine' 'Czech' 'Malaysian' 'Swiss' 'Belgian' 'Monegasque' 'Swedish'
 'Venezuelan' 'New Zealander' 'Chilean' 'Mexican' 'South African'
 'Liechtensteiner' 'Rhodesian' 'American-Italian' 'Uruguayan'
 'Argentine-Italian' 'Thai' 'East German' 'Russian' 'Indonesian' 'Chinese']


In [4]:
# Replacing nationality names for merge of lat and long later in code
nationality_replacements_driversdf = {
    "American-Italian": "Italian",
    "Argentine-Italian": "Italian",
    "East German": "German"
}

drivers_df['nationality'] = drivers_df['nationality'].replace(nationality_replacements_driversdf, regex=True)

# Filtering out rows that have 'Rhodesian' and 'Liechtensteiner' in the nationality column to avoid incorrect merging for lat and long
drivers_df = drivers_df[(drivers_df['nationality'] != 'Rhodesian') & (drivers_df['nationality'] != 'Liechtensteiner')]

drivers_df.head()

,driver_id,first_name,last_name,dob,nationality
0,1,Lewis,Hamilton,1985-01-07,British
1,2,Nick,Heidfeld,1977-05-10,German
2,3,Nico,Rosberg,1985-06-27,German
3,4,Fernando,Alonso,1981-07-29,Spanish
4,5,Heikki,Kovalainen,1981-10-19,Finnish


In [5]:
# Importing nationality.csv data into a dataframe
nationality_df = pd.read_csv('Resources/nationality.csv')

# Dropping "num_code","alpha_2_code" and "alpha_3_code" columns from nationality_df
nationality_df = nationality_df[['en_short_name', 'nationality']]

# Changing column names to be more intuitive for nationality_df
nationality_df=nationality_df.rename(columns = {'en_short_name': 'country_name'})
nationality_df.head()

,country_name,nationality
0,Afghanistan,Afghan
1,Åland Islands,Åland Island
2,Albania,Albanian
3,Algeria,Algerian
4,American Samoa,American Samoan


In [6]:
# Replacing nationality names for merge of lat and long later in code
nationality_replacements_natdf = {
    "British, UK": "British",
    "Dutch, Netherlandic": "Dutch",
    'Hungarian, Magyar': "Hungarian",
    "Monégasque, Monacan": "Monegasque",
    "New Zealand, NZ": "New Zealander",
    "American-Italian": "Italian",
    "Argentine-Italian": "Italian",
    "East German": "German"
}

# Replacing country names for merge of lat and long later in code
nationality_df['nationality'] = nationality_df['nationality'].replace(nationality_replacements_natdf, regex=True)


# Replacements dictionary for country names
country_replacements_natdf = {
    "Russian Federation": "Russia",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    r"Venezuela \(Bolivarian Republic of\)": "Venezuela",
    "United States Minor Outlying Islands": "United States",
    "United States of America": "United States",
    "Czech Republic": "Czechia"
}

# Replacing country names for merge of lat and long later in code
nationality_df['country_name'] = nationality_df['country_name'].replace(country_replacements_natdf, regex=True)

nationality_df.head()

,country_name,nationality
0,Afghanistan,Afghan
1,Åland Islands,Åland Island
2,Albania,Albanian
3,Algeria,Algerian
4,American Samoa,American Samoan


In [7]:
# Importing countries.csv data into a dataframe
countries_df = pd.read_csv('Resources/countries.csv')

# Dropping "country" column from countries_df
countries_df = countries_df[['latitude', 'longitude', 'name']]

# Changing column names to be more intuitive for nationality_df
countries_df=countries_df.rename(columns = {'name': 'country_name'})

countries_df

,latitude,longitude,country_name
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla
...,...,...,...
242,15.552727,48.516388,Yemen
243,-12.827500,45.166244,Mayotte
244,-30.559482,22.937506,South Africa
245,-13.133897,27.849332,Zambia


In [8]:
# Joining nationality_df into drivers_df through a left join on "nationality" to get "country_name"
merged_drivers_df = drivers_df.merge(nationality_df, on = 'nationality', how = 'left')

# Dropping the duplicate drivers that were created through the join
no_dupe_merged_drivers_df = merged_drivers_df.drop_duplicates(subset='driver_id', keep='first').reset_index(drop=True)

# Joining countries_df into no_dupe_merged_drivers_df through a left join on "country_name" to get "latitude" and "longtitude" which will be used for creating a map
cleaned_merged_drivers_df = no_dupe_merged_drivers_df.merge(countries_df, on = 'country_name', how = 'left')
cleaned_merged_drivers_df

,driver_id,first_name,last_name,dob,nationality,country_name,latitude,longitude
0,1,Lewis,Hamilton,1985-01-07,British,United Kingdom,55.378051,-3.435973
1,2,Nick,Heidfeld,1977-05-10,German,Germany,51.165691,10.451526
2,3,Nico,Rosberg,1985-06-27,German,Germany,51.165691,10.451526
3,4,Fernando,Alonso,1981-07-29,Spanish,Spain,40.463667,-3.749220
4,5,Heikki,Kovalainen,1981-10-19,Finnish,Finland,61.924110,25.748151
...,...,...,...,...,...,...,...,...
849,856,Nyck,de Vries,1995-02-06,Dutch,Netherlands,52.132633,5.291266
850,857,Oscar,Piastri,2001-04-06,Australian,Australia,-25.274398,133.775136
851,858,Logan,Sargeant,2000-12-31,American,United States,37.090240,-95.712891
852,859,Liam,Lawson,2002-02-11,New Zealander,New Zealand,-40.900557,174.885971


In [9]:
# Count null values in each column to check that merging was succesfull
null_counts = cleaned_merged_drivers_df.isnull().sum()
null_counts

driver_id       0
first_name      0
last_name       0
dob             0
nationality     0
country_name    0
latitude        0
longitude       0
dtype: int64

In [10]:
# Double check data types
cleaned_merged_drivers_df.dtypes

driver_id         int64
first_name       object
last_name        object
dob              object
nationality      object
country_name     object
latitude        float64
longitude       float64
dtype: object

In [11]:
Exporting the DataFrame "cleaned_merged_drivers_df" as a CSV file. 
cleaned_merged_drivers_df.to_csv("Resources/output_data_for_tables/drivers.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'Resources/output_data_for_tables/drivers.csv'

In [38]:
drivers_standing_df = pd.read_csv('Resources/driver_standings.csv')
drivers_standing_df=drivers_standing_df.rename(columns = {'driverId': 'driver_id'})
drivers_standing_grouped = drivers_standing_df.groupby('driver_id').sum()

drivers_standing_merge = pd.merge(drivers_df, drivers_standing_grouped, on='driver_id', how='left')
drivers_standing_merge = drivers_standing_merge[['driver_id','first_name', 'last_name','wins']]
drivers_standing_merge

,driver_id,first_name,last_name,wins
0,1,Lewis,Hamilton,1019.0
1,2,Nick,Heidfeld,0.0
2,3,Nico,Rosberg,269.0
3,4,Fernando,Alonso,346.0
4,5,Heikki,Kovalainen,8.0
...,...,...,...,...
849,856,Nyck,de Vries,0.0
850,857,Oscar,Piastri,0.0
851,858,Logan,Sargeant,0.0
852,859,Liam,Lawson,0.0
